In [1]:
import earthaccess
lon_min = -10
lat_min = 30
lon_max = 35
lat_max = 55

auth = earthaccess.login()
results = earthaccess.search_data(
    short_name="OCO2_L2_Lite_FP",
    temporal=("2023-05-05", "2024-01-02"),
    bounding_box=(-10, 30, 35, 55)
)

In [2]:
print(len(results))
print(results[0].data_links()[0])

397
https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2023/oco2_LtCO2_230504_B11100Ar_230705214613s.nc4


In [3]:
files = earthaccess.download(results[:5], '../data/raw')

QUEUEING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
import xarray as xr
datasets = [ xr.open_dataset(f'../data/raw/{file.name}', engine = 'netcdf4')
                for file in files]
datasets[0]

<xarray.Dataset> Size: 51MB
Dimensions:                 (sounding_id: 113830, epoch_dimension: 7,
                             vertices: 4, levels: 20, bands: 3, source_files: 22)
Coordinates:
  * sounding_id             (sounding_id) float64 911kB 2.023e+15 ... 2.023e+15
  * vertices                (vertices) int16 8B 1 2 3 4
  * levels                  (levels) int16 40B 1 2 3 4 5 6 ... 15 16 17 18 19 20
  * bands                   (bands) int16 6B 1 2 3
  * source_files            (source_files) object 176B 'oco2_L2StdGL_47004a_2...
Dimensions without coordinates: epoch_dimension
Data variables: (12/20)
    date                    (sounding_id, epoch_dimension) float32 3MB ...
    latitude                (sounding_id) float32 455kB ...
    longitude               (sounding_id) float32 455kB ...
    time                    (sounding_id) datetime64[ns] 911kB ...
    solar_zenith_angle      (sounding_id) float32 455kB ...
    sensor_zenith_angle     (sounding_id) float32 455kB ...
    ...                      ...
    xco2_uncertainty        (sounding_id) float32 455kB ...
    xco2_apriori            (sounding_id) float32 455kB ...
    pressure_levels         (sounding_id, levels) float32 9MB ...
    co2_profile_apriori     (sounding_id, levels) float32 9MB ...
    xco2_averaging_kernel   (sounding_id, levels) float32 9MB ...
    pressure_weight         (sounding_id, levels) float32 9MB ...
Attributes: (12/30)
    filter_function:                   oco2_xco2_quality_flag_b11_1
    bc_function:                       oco2_xco2_bias_correct_b11_1
    l2_reader:                         read_oco_l2_b11
    lite_definition_module:            lite_file_definition_oco2_b11_1
    met_files:                         ['oco2_L2MetGL_47004a_230504_B11008r_2...
    arp_files:                         oco2_ARP_46961_47052_v51_230608033922.h5
    ...                                ...
    Platform:                          OCO-2
    Sensor:                            OCO-2
    L2FullPhysicsExeVersion:           SVN-B11.0.05_rc2-38127
    title:                             OCO-2 L2FP Lite Output
    contact:                           Chris O'Dell: Christopher.ODell@colost...
    creation_date:                     Jul 05, 2023

In [5]:
year_from = 2024

combined = xr.concat(datasets, dim='sounding_id', join='outer')

xco2_ds = combined.where(combined["xco2_quality_flag"] == 0, drop=True)
# xco2_ds = xco2_ds.where(xco2_ds["time"].dt.year >= year_from, drop=True)
xco2_ds = xco2_ds.where(
        (xco2_ds["latitude"] >= lat_min)
        & (xco2_ds["latitude"] <= lat_max)
        & (xco2_ds["longitude"] >= lon_min)
        & (xco2_ds["longitude"] <= lon_max),
        drop=True,
    )

xco2_df = xco2_ds[["xco2", "time", "latitude", "longitude"]].to_dataframe().reset_index()
xco2_df = xco2_df.rename(columns={"latitude": "lat", "longitude": "lon"})
xco2_df.head()

,sounding_id,xco2,time,lat,lon
0,2.023050e+15,420.132721,2023-05-04 12:29:49.363016192,30.147869,9.560679
1,2.023050e+15,420.566528,2023-05-04 12:29:50.001998848,30.261932,9.551120
2,2.023050e+15,420.704559,2023-05-04 12:29:50.055991552,30.249044,9.528777
3,2.023050e+15,420.438202,2023-05-04 12:30:07.696008704,33.400700,9.068730
4,2.023050e+15,420.095276,2023-05-04 12:30:07.748995840,33.389999,9.037725


In [6]:
%reload_ext autoreload
%autoreload 2
from src.data.write import write_co2
from dotenv import load_dotenv
load_dotenv("../.env")

write_co2(xco2_ds, locally=True)

In [7]:
from src.data.load import load_co2

loaded = load_co2(raw=False)
loaded

<xarray.Dataset> Size: 5MB
Dimensions:                 (sounding_id: 10248, levels: 20,
                             epoch_dimension: 7, vertices: 4, bands: 3,
                             footprints: 8, frames: 122666, source_files: 54)
Coordinates:
  * sounding_id             (sounding_id) float64 82kB 2.023e+15 ... 2.023e+15
  * levels                  (levels) int16 40B 1 2 3 4 5 6 ... 15 16 17 18 19 20
  * vertices                (vertices) int16 8B 1 2 3 4
  * bands                   (bands) int16 6B 1 2 3
  * footprints              (footprints) int16 16B 1 2 3 4 5 6 7 8
  * frames                  (frames) int32 491kB 1 2 3 ... 122664 122665 122666
  * source_files            (source_files) object 432B 'oco2_L2StdGL_47004a_2...
Dimensions without coordinates: epoch_dimension
Data variables: (12/20)
    co2_profile_apriori     (sounding_id, levels) float32 820kB dask.array<chunksize=(10248, 20), meta=np.ndarray>
    date                    (sounding_id, epoch_dimension) float32 287kB dask.array<chunksize=(10248, 7), meta=np.ndarray>
    file_index              (sounding_id) float32 41kB dask.array<chunksize=(10248,), meta=np.ndarray>
    latitude                (sounding_id) float32 41kB dask.array<chunksize=(10248,), meta=np.ndarray>
    longitude               (sounding_id) float32 41kB dask.array<chunksize=(10248,), meta=np.ndarray>
    pressure_levels         (sounding_id, levels) float32 820kB dask.array<chunksize=(10248, 20), meta=np.ndarray>
    ...                      ...
    xco2_averaging_kernel   (sounding_id, levels) float32 820kB dask.array<chunksize=(10248, 20), meta=np.ndarray>
    xco2_qf_bitflag         (sounding_id) float64 82kB dask.array<chunksize=(10248,), meta=np.ndarray>
    xco2_qf_simple_bitflag  (sounding_id) float32 41kB dask.array<chunksize=(10248,), meta=np.ndarray>
    xco2_quality_flag       (sounding_id) float32 41kB dask.array<chunksize=(10248,), meta=np.ndarray>
    xco2_uncertainty        (sounding_id) float32 41kB dask.array<chunksize=(10248,), meta=np.ndarray>
    xco2_x2019              (sounding_id) float32 41kB dask.array<chunksize=(10248,), meta=np.ndarray>
Attributes: (12/30)
    Bias_Correction_land:              XCO2_Bias_Corrected = (XCO2_Raw + 0.82...
    Bias_Correction_landTG:            XCO2_Bias_Corrected = (XCO2_Raw + 0.77...
    Bias_Correction_oceanGL:           XCO2_Bias_Corrected = (XCO2_Raw + 0.25...
    Bias_Correction_oceanND:           XCO2_Bias_Corrected = (XCO2_Raw + 0.25...
    Bias_Uncertainty:                  Bias correction parameter uncertaintie...
    BuildId:                           B11.1.00
    ...                                ...
    identifier_product_doi_authority:  http://dx.doi.org/
    l2_reader:                         read_oco_l2_b11
    lite_definition_module:            lite_file_definition_oco2_b11_1
    met_files:                         ['oco2_L2MetGL_47004a_230504_B11008r_2...
    title:                             OCO-2 L2FP Lite Output
    warn_level_function:               None

In [45]:
loaded.chunksizes

Frozen({})